#### Audio

- 음성 비서
  - AI 콜센터, 음성 챗봇, 자동차 음성 비서 등
  - speech-to-speech(Realtime)
- Text-to-Speech(텍스트를 사람 목소리로 변환)
- Speech-to-Text
- Realtime API(audio=>audio,text), Transcription API(audio=>text), Speech API(text=>audio), chat.completions(예전거)
- Response API는 지원x

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [ ]:
import base64
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-audio",
    modalities=["text", "audio"],
    audio={"voice": "marin", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": "Is a golden retriever a good family dog?"
        }
    ]
)

print(completion.choices[0])

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("./audio/dog.wav", "wb") as f:
    f.write(wav_bytes)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=ChatCompletionAudio(id='audio_6998152888f88191b0b5533668d6f205', data='...', expires_at=1771578168, transcript="A golden retriever is often considered an excellent family dog. They're known for their friendly and gentle temperament, intelligence, and loyalty. They're great with children, usually very patient and easy to train. They do need regular exercise and enjoy being part of family activities. Overall, they're a loving and dependable breed, making them a popular choice for families."), function_call=None, tool_calls=None))


In [ ]:
# text => audio
speech = client.audio.speech.create(
    model="gpt-4o-mini-tts", voice="ballad", input="안녕하세요"
)

with open("./audio/tts.wav", "wb") as f:
    f.write(speech.read())

In [4]:
# audio => text
with open("./audio/obama.mp3", "rb") as f:
    transcript = client.audio.transcriptions.create(
        model="gpt-4o-mini-transcribe",
        file=f,
    )

print(transcript.text)

Hello, Chicago! It's good to be home. Tonight, it's my turn to say thanks. And every day I have learned from you. You made me a better President, and you made me a better man. I can't do that. Where I learned that change only happens when ordinary people get involved, and they get engaged, and they come together to demand it. After eight years as your President, I still believe that. In ten days, the world will witness a hallmark of our democracy. No, no, no, no, no. The peaceful transfer of power from one freely elected president to the next. I committed to President-elect Trump that my administration would ensure the smoothest possible transition, just as President Bush did for me. Because, after all, we remain the wealthiest, most powerful, and most respected nation on Earth. Michelle, for the past 25 years, you have not only been my wife and mother of my children, you have been my best friend. You have made me proud, and you have made the country proud. My fellow Americans, it has 

In [ ]:
import base64
import requests
from openai import OpenAI

client = OpenAI()

# Fetch the audio file and convert it to a base64 encoded string
url = "https://cdn.openai.com/API/docs/audio/alloy.wav"
response = requests.get(url)

response.raise_for_status()  # status에 따라 응답(에러)

wav_data = response.content  # byte 코드로 가져오기
encoded_string = base64.b64encode(wav_data).decode("utf-8")

completion = client.chat.completions.create(
    model="gpt-audio",
    modalities=["text", "audio"],  # 응답 구조 : text+audio
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What is in this recording?"},
                {
                    "type": "input_audio",
                    "input_audio": {"data": encoded_string, "format": "wav"},
                },
            ],
        },
    ],
)

print(completion.choices[0].message)

# completion text로 출력
print(f"답변 : {completion.choices[0].message.audio.transcript}")

# audio로 출력
audio_base64 = completion.choices[0].message.audio.data
audio_bytes = base64.b64decode(audio_base64)

with open("./audio/response.wav", "wb") as fw:
    fw.write(audio_bytes)